<a href="https://colab.research.google.com/github/i-SanMartin/FastAI/blob/main/04_MNIST_Basics/Network_Exercise_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqq fastbook fastai==2.2.5
import fastbook

     |████████████████████████████████| 727kB 5.4MB/s 
     |████████████████████████████████| 194kB 8.3MB/s 
     |████████████████████████████████| 51kB 4.3MB/s 
     |████████████████████████████████| 1.2MB 8.8MB/s 
     |████████████████████████████████| 12.8MB 293kB/s 
     |████████████████████████████████| 776.8MB 22kB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.


In [2]:
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [3]:
path = untar_data(URLs.MNIST)

In [4]:
Path.BASE_PATH = path

In [5]:
categories = tuple(x for x in range(0,10))
categories

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)

In [6]:
#Training data
train_x = torch.tensor([])
train_y = torch.tensor([])

for x in categories:
  number_imgs = (path/'training'/str(x)).ls().sorted()
  zeros_tensor =  [tensor(Image.open(img)) for img in number_imgs]
  stacked_zeros = torch.stack(zeros_tensor)
  train_x = torch.cat([train_x,stacked_zeros])
  y = torch.tensor([x]*len(number_imgs))
  train_y = torch.cat([train_y,y])

In [7]:
#Validation data
valid_x = torch.tensor([])
valid_y = torch.tensor([])

for x in categories:
  number_imgs = (path/'testing'/str(x)).ls().sorted()
  zeros_tensor =  [tensor(Image.open(img)) for img in number_imgs]
  stacked_zeros = torch.stack(zeros_tensor)
  valid_x = torch.cat([valid_x,stacked_zeros])
  y = torch.tensor([x]*len(number_imgs))
  valid_y = torch.cat([valid_y,y])

In [8]:
#Reshape data
train_x = train_x.view(-1,28*28).float()/255
valid_x = valid_x.view(-1,28*28).float()/255
train_y = train_y.unsqueeze(1).long()
valid_y = valid_y.unsqueeze(1).long()
train_x.shape, train_y.shape, valid_x.shape, valid_y.shape

(torch.Size([60000, 784]),
 torch.Size([60000, 1]),
 torch.Size([10000, 784]),
 torch.Size([10000, 1]))

In [12]:
#Create datasets
batch_size=256
dset = list(zip(train_x,train_y))
dl = DataLoader(dset, batch_size=batch_size)
valid_dset = list(zip(valid_x,valid_y))
valid_dl = DataLoader(dset, batch_size=batch_size)
dls = DataLoaders(dl, valid_dl)

In [10]:
#Optimizer
class Sgd():
  def __init__(self, parameters, lr):
    self.parameters = list(parameters)
    self.lr = lr
  def step(self):
    for p in self.parameters:
      p.data -= p.grad * self.lr
  def zero_grad(self):
    for p in self.parameters:
      p.grad = None

In [45]:
#Neural network
simple_net = nn.Sequential(
    nn.Linear(28*28,50),
    nn.ReLU(),
    nn.Linear(50,40),
    nn.ReLU(),
    nn.Linear(40,30),
    nn.ReLU(),
    nn.Linear(30,len(categories)),
    nn.Sigmoid()
)

In [13]:
#Metric function
def batch_accuracy(x,y):
    pred = simple_net(x)
    pred_class = torch.tensor([torch.argmax(i) for i in pred])
    correct = (pred_class == y.squeeze(1)).float().sum()
    return correct

In [37]:
#Learner
class learner():
  def __init__(self, Dls, model, optimize, loss_func, metrics):
    self.dls = Dls
    self.model = model
    self.opt = optimize
    self.loss_func = loss_func
    self.metrics = metrics

  def train(self, epochs, lr):
    self.Opt = self.opt(self.model.parameters(), lr)
    metric = self.validate_epoch(valid_dl).item()
    loss = self.loss_func(self.model(valid_x),valid_y.squeeze(1))
    for i in range(epochs):
      print("Epoch: {}   Metric:  {}   Loss:   {:0.4f}".format(i, metric, loss))
      self.train_epoch(dl)
      metric = self.validate_epoch(valid_dl).item()
      loss = self.loss_func(self.model(valid_x),valid_y.squeeze(1))
    print("Epoch: {}   Metric:  {}   Loss:   {:0.4f}".format(i, metric, loss))

  def predict(x):
    pred = torch.argmax(self.model(x))
    return(pred)

  def train_epoch(self, data):
    for dx, dy in data:
      self.calc_grad(dx,dy)
      self.Opt.step()
      self.Opt.zero_grad()

  def validate_epoch(self, data):
    accs = torch.tensor([0.])
    for xb,yb in data:
      accs += self.metrics(xb,yb)
    return accs/len(data.dataset)

  def calc_grad(self,x,y): 
    pred = self.model(x)
    loss = self.loss_func(pred,y.squeeze(1))
    loss.backward()
    return loss
  

In [46]:
learn = learner(dls, simple_net, Sgd, nn.CrossEntropyLoss(), batch_accuracy)

In [47]:
#Intentar conseguir que llegue a una metrica de 0.8 en 20 
learn.train(50, 0.1)

Epoch: 0   Metric:  0.05608333274722099   Loss:   2.3038
Epoch: 1   Metric:  0.268449991941452   Loss:   2.2897
Epoch: 2   Metric:  0.46853333711624146   Loss:   2.1100
Epoch: 3   Metric:  0.5612833499908447   Loss:   1.8231
Epoch: 4   Metric:  0.6464499831199646   Loss:   1.7334
Epoch: 5   Metric:  0.7162833213806152   Loss:   1.6884
Epoch: 6   Metric:  0.7525833249092102   Loss:   1.6521
Epoch: 7   Metric:  0.7799000144004822   Loss:   1.6271
Epoch: 8   Metric:  0.7997833490371704   Loss:   1.6101
Epoch: 9   Metric:  0.8077666759490967   Loss:   1.5959
Epoch: 10   Metric:  0.828416645526886   Loss:   1.5850
Epoch: 11   Metric:  0.8360000252723694   Loss:   1.5768
Epoch: 12   Metric:  0.848716676235199   Loss:   1.5705
Epoch: 13   Metric:  0.8714500069618225   Loss:   1.5644
Epoch: 14   Metric:  0.8817333579063416   Loss:   1.5606
Epoch: 15   Metric:  0.8890500068664551   Loss:   1.5552
Epoch: 16   Metric:  0.8997166752815247   Loss:   1.5508
Epoch: 17   Metric:  0.9053833484649658   

In [51]:
simple_net(train_x[20000])

tensor([1.9483e-14, 4.7293e-10, 1.0334e-04, 1.0000e+00, 6.0645e-30, 3.5568e-11, 2.7157e-26, 2.6243e-10, 6.9453e-07, 2.4112e-12], grad_fn=<SigmoidBackward>)